In [3]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, make_scorer, f1_score
from collections import Counter
import scipy.stats as stats # for the breakpoints in SAX
from scipy.stats import norm
from Symbol import SYMBOLS
from distances import MINDIST
from SAX_transf import SAX_transform
import warnings 
from dtw import dtw
from sklearn.model_selection import GridSearchCV
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from tslearn.utils import to_time_series_dataset
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc


warnings.filterwarnings('ignore')
import os
import aeon
from aeon.datasets import load_classification

In [2]:
train_path = os.path.join(os.getcwd(), "datasets/classification/ECG200/ECG200_TRAIN.ts")
test_path = os.path.join(os.getcwd(), "datasets/classification/ECG200/ECG200_TEST.ts")

X_train = pd.read_csv(train_path, 
                      sep=",", 
                      header=None
                      )

X_train.columns = list(X_train.columns[:-1]) + ['label']
x_train, y_train = X_train.iloc[:, :-1], X_train["label"]

y_train = y_train.replace(-1, 0)

X_test = pd.read_csv(test_path, 
                      sep=",", 
                      header=None
                      )

X_test.columns = list(X_test.columns[:-1]) + ['label']
x_test, y_test = X_test.iloc[:, :-1], X_test["label"]

y_test = y_test.replace(-1, 0)

In [5]:
ts_1 = x_train.iloc[0,:] # first time series, length 96
num_seg = 10
alphabet_size = 5
sax = SAX_transform(ts_1, num_seg, alphabet_size)


In [18]:
l = sax.segment_time_series(ts_1)

# sax.tsax_paa(ts_1)

find_key_points(l[2])

[(0, -0.35035721), (7, -1.044091), (5, -1.1539497), (8, -1.2020312)]

In [11]:
def find_key_points(time_series):
    """
    Détecte les points clés (start, MP, MD, end) dans une série temporelle.
    - Start : Premier point.
    - End : Dernier point.
    - MP : Point avec la plus grande distance verticale au-dessus de la ligne de tendance.
    - MD : Point avec la plus grande distance verticale en dessous de la ligne de tendance.

    Arguments :
        time_series : list of float, la série temporelle (les valeurs y au fil du temps).

    Retourne :
        list : Une liste de quatre points clés sous la forme [(x_start, y_start), (x_MP, y_MP), (x_MD, y_MD), (x_end, y_end)].
    """
    # Étape 1 : Convertir la série temporelle en points (x, y)
    points = [(i, y) for i, y in enumerate(time_series)]  # i = temps, y = valeur
    
    # Étape 2 : Initialiser les points start et end
    start = points[0]
    end = points[-1]
    
    # Étape 3 : Calculer la ligne de tendance entre start et end
    def vertical_distance(p, start, end):
        """
        Calcule la distance verticale (VD) d'un point p à la ligne de tendance entre start et end.
        """
        x1, y1 = start
        x2, y2 = end
        x, y = p

        # Équation de la droite (ligne de tendance)
        if x2 - x1 == 0:  # Éviter la division par zéro pour une ligne verticale
            return abs(x - x1)
        
        slope = (y2 - y1) / (x2 - x1)
        intercept = y1 - slope * x1
        y_trend = slope * x + intercept  # Valeur prédite par la ligne de tendance
        
        return y - y_trend  # Distance verticale (positive si au-dessus, négative si en dessous)
    
    # Étape 4 : Rechercher MP (Maximum Peak) et MD (Maximum Dip)
    max_positive_distance = float('-inf')
    max_negative_distance = float('inf')
    MP = None
    MD = None

    for point in points[1:-1]:  # Ignorer start et end
        distance = vertical_distance(point, start, end)
        if distance > max_positive_distance:
            max_positive_distance = distance
            MP = point
        if distance < max_negative_distance:
            max_negative_distance = distance
            MD = point

    # Étape 5 : Gérer les cas où MP ou MD n'existent pas (grâce à une initialisation par défaut)
    if MP is None:
        MP = start  # Par défaut, MP est le point de départ si aucun pic n'existe
    if MD is None:
        MD = start  # Par défaut, MD est le point de départ si aucune dépression n'existe

    # Étape 6 : Retourner les quatre points clés
    return [start, MP, MD, end]